In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys
import os

# Get the parent directory of the current notebook
notebook_dir = os.getcwd()  # Use current working directory instead of __file__
parent_dir = os.path.dirname(notebook_dir)

# Add `src` to sys.path
sys.path.append(os.path.join(parent_dir, "src"))

In [20]:
from tests.loader import Loader
game1 = Loader("game1")
rules = game1.getRules()

In [21]:
from MAGS import MAGS

In [22]:
mags = MAGS()

In [23]:
# split rules by double newlines
rules_split = rules.split("\n\n")
print(len(rules_split))

9


In [24]:
for index, document in enumerate(rules_split):
    await mags.add_text_to_LTM_Semantic([document])

In [25]:
print(len(mags.long_term_memory.nodes))

9


In [26]:
print(await mags.query("Give me a high level overview of the game"))

{
   "original": "Give me a high level overview of the game",
   "rephrased": [
     "Game summary",
     "Overview of gameplay mechanics",
     "Top-level description of the game"
   ]
}
You are an expert at answering questions for a user. You have a content-retrieval system memory that can be used to answer questions. In the previous step you used
this memory to recall the following bits of information. Using this information, any past messages, your knowledge, and the user's question, answer the user's question.
as best as you can.

If the user gives you new information, you may add it to your memory. Be careful when adding new memory, 
since you may exceed memory limits. This memory SHOULD NOT be chat history, but rather new information
about the game or 'breakthroughs' you had that you did not know about before. Be stingy with your new memory.
It is completely ok and expected in most cases to add 0 (zero) new memory. For example,
add information about changes to the rules or struc

In [28]:
print(await mags.query("The Blue D6 Dice (B6) now only has 1 side with the number 1. Update your memory."))

{"original": "The Blue D6 Dice (B6) now only has 1 side with the number 1", "rephrased": ["A single-sided die labeled '1'", "A B6 die with only one face showing the number 1", "What is a modified version of the Blue D6 Dice with only one numbered side"]}
You are an expert at answering questions for a user. You have a content-retrieval system memory that can be used to answer questions. In the previous step you used
this memory to recall the following bits of information. Using this information, any past messages, your knowledge, and the user's question, answer the user's question.
as best as you can.

If the user gives you new information, you may add it to your memory. Be careful when adding new memory, 
since you may exceed memory limits. This memory SHOULD NOT be chat history, but rather new information
about the game or 'breakthroughs' you had that you did not know about before. Be stingy with your new memory.
It is completely ok and expected in most cases to add 0 (zero) new memor

In [29]:
print(len(mags.long_term_memory.nodes))

10


In [30]:
print(await mags.query("How many ides does the Blue D6 Dice (B6) have?"))

{
    "original": "How many ideas does the Blue D6 Dice (B6) have?",
    "rephrased": [
        "What are the concepts or themes associated with a Blue D6 Dice (B6)?",
        "Can you retrieve information about the characteristics of a Blue D6 Dice (B6)?",
        "What keywords or tags might be used to describe a Blue D6 Dice (B6) in a data set?"
    ]
}
You are an expert at answering questions for a user. You have a content-retrieval system memory that can be used to answer questions. In the previous step you used
this memory to recall the following bits of information. Using this information, any past messages, your knowledge, and the user's question, answer the user's question.
as best as you can.

If the user gives you new information, you may add it to your memory. Be careful when adding new memory, 
since you may exceed memory limits. This memory SHOULD NOT be chat history, but rather new information
about the game or 'breakthroughs' you had that you did not know about before. 

## Problem Statement and Implementation in ReadME.md

## Validation
Test cases above. Seems like the impelementation does work as intended,
but the performance is not spectacular. A lot of the times the model
will output invalid JSON or not follow instructions properly. We may need to upgrade
to a larger model or use instruct models to better follow this. We are still
working on the RL algorithm due to issues with getting access and the math.

During the infernece memory spiked, but I only allowed half my cores to be used, so it seems stable. When we use a GPU we likely need to better watch our resource allocaiton.

We added error handling for any issues, but have not yet tested servere edge cases. We tested a few edge cases but overall it seems to not be too bad.

## Next Steps

We need to finish the RL algorithm / agent so that we don't have to use JSON and it is actually encouraged to use memory in the *right cases*. We also need to try to use a larger model. Finally, we need to be able to implement the full working graphs including consolidaiton, short term memory, edge decay, etc...